In [1]:
import numpy as np

In [138]:
class CNN:
    def __init__(self):
        pass

    def conv(self, image, kernal, featureMapShape,
             f=lambda x: x if(x > 0) else 0):
        if (featureMapShape):
            featureMap = np.zeros(featureMapShape)
            rng = featureMapShape[0]
        else:
            rng = int(len(image) - np.ceil(len(kernal)/2))
            featureMap = np.zeros((rng, rng))
        for i in range(rng):
            for j in range(rng):
                mbuff = image[j: j+len(kernal),
                              i: i+len(kernal)] * kernal 
                featureMap[j][i] = f(np.sum(mbuff))
        return featureMap
    
    def pooling(self, featureMap, box=3, stride=3, 
                f=lambda x:np.max(x)):
        rng = int((np.ceil(len(featureMap) / box)) * box)
        rng2 =int(rng/stride)
        pool = np.zeros((rng2, rng2))
        for i in range(rng2):
            for j in range(rng2):
                istride = i * stride
                jstride = j * stride
                pool[i][j] = f(featureMap[
                               istride: istride+box,
                               jstride: jstride+box])
        return pool
        
    def resizeImage(self, image, size, xbuff=0, ybuff=0):
        assert len(image) < size[0], "ERROR: resizeImage()"
        assert len(image[0]) < size[1], "ERROR: resizeImage()"
        canv = np.zeros(size)
        canv[ybuff:len(image)+ybuff,
             xbuff:len(image[0])+xbuff] += image
        return canv

In [1]:
class NeuronLayer:
    def __init__(self, count, nextCount, first=False):
        self.count = count
        self.nextCount = nextCount
        self.neurons = np.zeros((self.count, 1))
        if(first):
            self.errors = np.zeros((self.count, 1))
        if(nextCount != 0):
            self.errors = np.zeros((self.nextCount, self.count))
            self.synapses = np.random.rand(self.nextCount, self.count)

class NeuralNetwork:
    def __init__(self):
        self.layers = 0
        self.network = list()

    def initNetwork(self, networkStruct):
        self.layers = len(networkStruct)
        self.network.append(NeuronLayer(networkStruct[0], networkStruct[1], True))
        for i in range(self.layers-2):
            self.network.append(NeuronLayer(networkStruct[i+1], networkStruct[i+2]))
        self.network.append(NeuronLayer(networkStruct[-1], 0))

    def feedForward(self, data):
        self.network[0].neurons = data.reshape(self.network[0].count, 1)
        for i in range(self.layers-1):
            if(i != self.layers-2):
                self.network[i+1].neurons = self.sigmoid(np.dot(self.network[i].synapses, self.network[i].neurons))
            else:
                self.network[i+1].neurons = self.softmax(np.dot(self.network[i].synapses, self.network[i].neurons))

    def backpropagation(self, target, learningRate):
        self.network[-1].errors = target - self.network[-1].neurons
        for i in reversed(range(self.layers-1)):
            if(i != 0):
                self.network[i].errors = np.dot(self.network[i].synapses.T, self.network[i+1].errors)
            self.network[i].synapses += learningRate * np.dot(self.network[i+1].errors, self.network[i].neurons.T)

    def train(self, trainLabels, trainData, testLabels, testData, epochs, learningRate):
        for i in range(epochs):
            print('\t-- Epoch {}'.format(i+1))
            for label, data in zip(trainLabels, trainData):
                target = self.oneHotEncode(label-1)
                self.feedForward(data)
                self.backpropagation(target, learningRate)
            accuracy = self.test(testLabels, testData)
            print('Accuracy = {0:.2f}%'.format(accuracy*100))

    def test(self, labels, testData):
        correct = 0
        for i, (label, data) in enumerate(zip(labels, testData)):
            self.feedForward(data)
            bestIndex = np.argmax(self.network[-1].neurons)
            if (label == bestIndex+1):
                correct += 1
        return correct / len(labels)

    def oneHotEncode(self, index):
        vect = np.zeros((self.network[-1].count, 1))
        vect[index] = 1
        return vect

    def sigmoid(self, A):
        return 1 / (1 + np.exp(-A))

    def softmax(self, A):
        e = np.exp(A - np.max(A))
        return e / e.sum()

In [141]:
print('Image')
image = np.array([[0,0,0,0,0,0,0],
                  [0,1,0,0,0,1,0],
                  [0,0,0,0,0,0,0],
                  [0,0,0,1,0,0,0],
                  [0,1,0,0,0,1,0],
                  [0,0,1,1,1,0,0],
                  [0,0,0,0,0,0,0]])
print(image)
print('Kernal')
kernal = np.array([[0,0,1],
                   [1,0,0],
                   [0,1,1]])
print(kernal)
cnn = CNN()
print('Feature Map')
featureMap = cnn.conv(image, kernal, (5,5))
print(featureMap)
print('Pooling')
pool = cnn.pooling(featureMap, 2, 2)
print(pool)

Image
[[0 0 0 0 0 0 0]
 [0 1 0 0 0 1 0]
 [0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 1 0 0 0 1 0]
 [0 0 1 1 1 0 0]
 [0 0 0 0 0 0 0]]
Kernal
[[0 0 1]
 [1 0 0]
 [0 1 1]]
Feature Map
[[0. 1. 0. 0. 0.]
 [0. 1. 1. 1. 0.]
 [1. 0. 0. 2. 1.]
 [1. 4. 2. 1. 0.]
 [0. 0. 1. 2. 1.]]
Pooling
[[1. 1. 0.]
 [4. 2. 1.]
 [0. 2. 1.]]
